<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/mesa_ABM_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit mesa


In [41]:
!wget -qO- icanhazip.com

34.74.20.153


In [47]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.20.153:8501

your url is: https://shiny-shoes-send.loca.lt
  Stopping...
^C


In [ ]:

import mesa
import pandas as pd
import random
import streamlit as st
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid

# Данные
data = pd.DataFrame({
    "sku": ["sku1", "sku2", "sku3", "sku4", "sku5", "sku6", "sku7", "sku8", "sku9", "sku10"],
    "share_vol": [0.1, 0.2, 0.15, 0.05, 0.25, 0.3, 0.07, 0.04, 0.12, 0.08],
    "price": [5, 4, 6, 8, 3, 7, 2, 10, 9, 1],
    "sales_vol": [100, 150, 120, 80, 200, 250, 70, 40, 90, 60],
})

# Агенты-потребители
class Consumer(Agent):
    """Агент-потребитель."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        # Агенты выбирают SKU для покупки на основе веса
        choice = random.choices(self.model.skus, weights=self.model.weights, k=1)[0]
        self.model.sales_history[choice] += 1  # Обновляем историю продаж

        # Передвижение агента по сетке
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_moves)
        self.model.grid.move_agent(self, new_position)

# Модель рынка
class Market(Model):
    """Модель рынка с агентами."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, True)  # Создаем сетку
        self.skus = data["sku"].tolist()
        self.weights = data["share_vol"].tolist()

        # История продаж для каждого SKU
        self.sales_history = {sku: 0 for sku in self.skus}
        self.sales_over_time = {sku: [] for sku in self.skus}

        # Добавляем агентов на сетку
        for i in range(self.num_agents):
            a = Consumer(i, self)
            self.schedule.add(a)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

    def step(self):
        # Один шаг модели - агенты совершают покупки и перемещаются
        self.schedule.step()
        # Обновляем историю продаж для каждого SKU
        for sku in self.skus:
            self.sales_over_time[sku].append(self.sales_history[sku])

# Streamlit интерфейс
def run_app():
    st.title("Зависимость SKU от продаж во времени")

    # Инициализация модели: 100 агентов на сетке 10x10
    model = Market(100, 10, 10)

    # Симуляция на 30 дней
    days = 30
    for day in range(days):
        model.step()

    # Ползунок для выбора дня
    selected_day = st.slider("Выбор дня", 0, days - 1, 0)

    # Построение графика изменения продаж на выбранный день
    fig, ax = plt.subplots()
    for sku in model.skus:
        ax.plot(range(selected_day + 1), model.sales_over_time[sku][:selected_day + 1], label=sku)

    ax.set_xlabel("День")
    ax.set_ylabel("Обьем продаж")
    ax.legend()

    # Отображение графика в Streamlit
    st.pyplot(fig)

if __name__ == "__main__":
    run_app()
